<a href="https://colab.research.google.com/github/mvadrev/Titanic/blob/main/Assignment1_Titanic_binary_logistic_regression_Mukund.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
#import pytorch and pandas
import torch
import pandas as pd
#Set manual seed for reproducibility 
torch.manual_seed(1)


In [104]:
#USe pandas to read a csv and print the head
dataset = pd.read_csv('train.csv')
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [105]:
#Preprocess data
#Getting mean age 
avgAge = dataset['Age'].mean()
dataset.info()
dataset['Age'] = dataset['Age'].fillna(avgAge)
dataset.info()
dataset['Age'] = dataset['Age']/avgAge
dataset['Fare'] = dataset['Fare']/(dataset['Fare'].mean())
dataset['Sex'] = dataset['Sex'].map({'male': 0, 'female': 1})
dataset['Embarked'] = dataset['Embarked'].map({'S': 0, 'C': 1, 'Q': 2, 'nan': 3})
dataset['Embarked'] = dataset['Embarked'].fillna(0)
print(dataset['Embarked'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived

In [106]:
cleanedDataset = dataset[['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked', 'Survived']]
cleanedDataset.head()

,Pclass,Sex,Age,SibSp,Fare,Embarked,Survived
0,3,0,0.740763,1,0.225126,0.0,0
1,1,1,1.279499,1,2.213478,1.0,1
2,3,1,0.875447,0,0.246086,0.0,1
3,1,1,1.178486,1,1.648853,0.0,1
4,3,0,1.178486,0,0.249967,0.0,0


In [107]:
xDataset = cleanedDataset[['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked',]].values



In [108]:
yDataset = dataset[['Survived']].values
 

In [109]:
# Splitting data for cross validation
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(xDataset, yDataset, stratify=yDataset, random_state = 7)

In [110]:
from   torch.utils.data import Dataset
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

class ds(Dataset):
    def __init__(self):
        self.x = torch.from_numpy(x_train)
        self.y = torch.from_numpy(y_train).view(-1,1)

    def __getitem__(self, i):        
        return self.x[i],self.y[i]
    
    def __len__(self):
        return len(self.x)

In [207]:
train_ds = ds() 
trainLoader = DataLoader(train_ds,)

In [208]:
#Define model
from torch import nn
import torch.nn.functional as F


class Titanic(nn.Module):
  def __init__(self,inputFeatures,outPutFeatures):
    super(Titanic,self).__init__()
    self.fc1 = torch.nn.Linear(inputFeatures,100)
    self.fc2 = torch.nn.Linear(100,50)
    self.fc3 = torch.nn.Linear(50,20)
    self.fc4 = torch.nn.Linear(20,1)
  

  def forward(self,x): 
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    return torch.sigmoid(x)

In [209]:
model = Titanic(6,1)
print(model)


Titanic(
  (fc1): Linear(in_features=6, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=1, bias=True)
)


In [210]:
import torch.optim as optim
# Loss function is binary cross entropy loss as this is a logistic regression problem
criterion = nn.BCELoss()
lr = 0.002
epochs = 1
#Using stoichastic gradient descent
optimizer = optim.SGD(model.parameters(),lr=lr,)

In [211]:
for epoch in range(70):
  for batchidx ,(inputs,labels) in enumerate(trainLoader):
    optimizer.zero_grad()
    outputs = model(inputs.float())
    loss = criterion(outputs, labels.float())
    loss.backward()
    optimizer.step()
  print(loss.item())

0.7981538772583008
0.7642055749893188
0.676468551158905
0.5562074780464172
0.4341519773006439
0.34252509474754333
0.27419981360435486
0.22478243708610535
0.18995647132396698
0.16624917089939117
0.15099740028381348
0.1404844969511032
0.13033345341682434
0.1200583204627037
0.10992493480443954
0.10146219283342361
0.093556709587574
0.08640662580728531
0.08086296916007996
0.0777634009718895
0.07403681427240372
0.07091446965932846
0.0680031105875969
0.06543862074613571
0.06273940950632095
0.06019007787108421
0.05807561054825783
0.056250058114528656
0.054425884038209915
0.05336599424481392
0.05164164677262306
0.05045843869447708
0.049251895397901535
0.04708411544561386
0.04566967114806175
0.044540178030729294
0.043449126183986664
0.042368337512016296
0.041394881904125214
0.04047419875860214
0.03950066119432449
0.03852707892656326
0.037802886217832565
0.036958664655685425
0.03622417151927948
0.035344354808330536
0.03455397114157677
0.03396984189748764
0.033106666058301926
0.032528676092624664


In [184]:
testDataSet = pd.read_csv('test.csv')
#Preprocess test.csv data
avgFare = testDataSet['Fare'].mean()
print("The average Fare is",avgFare)
avgAge = testDataSet['Age'].mean()
print("The average age is",avgAge)
print(testDataSet.info())
testDataSet['Age'] = testDataSet['Age'].fillna(avgAge)
testDataSet.info()
testDataSet['Age'] = testDataSet['Age']/avgAge
testDataSet['Fare'] = testDataSet['Fare']/avgFare
testDataSet['Sex'] = testDataSet['Sex'].map({'male': 0, 'female': 1})
testDataSet['Embarked'] = testDataSet['Embarked'].map({'S': 0, 'C': 1, 'Q': 2, 'nan': 3})
testDataSet['Embarked'] = testDataSet['Embarked'].fillna(0)
print(testDataSet['Embarked'].unique())
testDataSubSet = testDataSet[['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked',]]
print(testDataSubSet [[ 'Fare']])
testDataSubSet [[ 'Age']]
testDataSet['Fare'] = testDataSet['Fare'].fillna(0)



The average Fare is 35.6271884892086
The average age is 30.272590361445783
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId 

In [185]:
testDataSubSet
#Normalized age with mean age and fare with mean fare for faster training

,Pclass,Sex,Age,SibSp,Fare,Embarked
0,3,0,1.139645,0,0.219754,2
1,3,1,1.552560,1,0.196479,0
2,2,0,2.048057,0,0.271913,2
3,3,0,0.891896,0,0.243143,0
4,3,1,0.726730,1,0.344891,0
...,...,...,...,...,...,...
413,3,0,1.000000,0,0.225951,0
414,1,1,1.288294,0,3.056654,1
415,3,0,1.271778,0,0.203496,0
416,3,0,1.000000,0,0.225951,0


In [186]:
testDataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 36.0+ KB


In [187]:
class Testds(Dataset):
    def __init__(self):
        self.x = torch.from_numpy(testDataSubSet.values)

    def __getitem__(self, i):        
        return self.x[i]
    
    def __len__(self):
        return len(self.x)

In [188]:
validatioDS = Testds()
validationLoader = DataLoader(validatioDS)
len(validationLoader)

418

In [212]:
count = 0
total = len(validationLoader)
passengerList =testDataSet['PassengerId']
print(total)
submission = [['PassengerId', 'Survived']]
for i,(data) in enumerate(validationLoader):
  with torch.no_grad():
    output_ = model(data.float())
    print("The output is", output_)
    if (output_ >= 1): 
      output_ = 1
      submission.append([passengerList[i], output_])
    else:
      output_ = 0

      submission.append([passengerList[i], output_])


418
The output is tensor([[0.0964]])
The output is tensor([[0.1896]])
The output is tensor([[0.0666]])
The output is tensor([[0.1176]])
The output is tensor([[0.4499]])
The output is tensor([[0.1831]])
The output is tensor([[0.7489]])
The output is tensor([[0.2252]])
The output is tensor([[0.7054]])
The output is tensor([[0.1114]])
The output is tensor([[0.1089]])
The output is tensor([[0.2870]])
The output is tensor([[0.9762]])
The output is tensor([[0.0998]])
The output is tensor([[0.9744]])
The output is tensor([[0.9630]])
The output is tensor([[0.1485]])
The output is tensor([[0.1529]])
The output is tensor([[0.3556]])
The output is tensor([[0.3023]])
The output is tensor([[0.3072]])
The output is tensor([[0.2067]])
The output is tensor([[0.9676]])
The output is tensor([[0.5568]])
The output is tensor([[0.8979]])
The output is tensor([[0.0635]])
The output is tensor([[0.9808]])
The output is tensor([[0.1449]])
The output is tensor([[0.3352]])
The output is tensor([[0.0931]])
The ou

In [213]:
submission

[['PassengerId', 'Survived'],
 [892, 0],
 [893, 0],
 [894, 0],
 [895, 0],
 [896, 0],
 [897, 0],
 [898, 0],
 [899, 0],
 [900, 0],
 [901, 0],
 [902, 0],
 [903, 0],
 [904, 0],
 [905, 0],
 [906, 0],
 [907, 0],
 [908, 0],
 [909, 0],
 [910, 0],
 [911, 0],
 [912, 0],
 [913, 0],
 [914, 0],
 [915, 0],
 [916, 0],
 [917, 0],
 [918, 0],
 [919, 0],
 [920, 0],
 [921, 0],
 [922, 0],
 [923, 0],
 [924, 0],
 [925, 0],
 [926, 0],
 [927, 0],
 [928, 0],
 [929, 0],
 [930, 0],
 [931, 0],
 [932, 0],
 [933, 0],
 [934, 0],
 [935, 0],
 [936, 0],
 [937, 0],
 [938, 0],
 [939, 0],
 [940, 0],
 [941, 0],
 [942, 0],
 [943, 0],
 [944, 0],
 [945, 0],
 [946, 0],
 [947, 0],
 [948, 0],
 [949, 0],
 [950, 0],
 [951, 0],
 [952, 0],
 [953, 0],
 [954, 0],
 [955, 0],
 [956, 0],
 [957, 0],
 [958, 0],
 [959, 0],
 [960, 0],
 [961, 0],
 [962, 0],
 [963, 0],
 [964, 0],
 [965, 0],
 [966, 0],
 [967, 0],
 [968, 0],
 [969, 0],
 [970, 0],
 [971, 0],
 [972, 0],
 [973, 0],
 [974, 0],
 [975, 0],
 [976, 0],
 [977, 0],
 [978, 0],
 [979, 0],
 [

In [214]:
import csv
with open('submission.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(submission)